# Baseline Word RNN
***

### Data:
8 Dr. Seuss Books

Word-Sequence Length = 25

### Model: 
2-layer LSTM, 400 hidden states, dropout ratio = 0.2

### Training:
1 epoch, batch size of 100

---

### Results: 

congratulations! today is your day. you're off to great places! you're off and away!  you have brain                                                                                                                                                                                                        


---

## 1. Import Text Data

In [1]:
total = ""
data = ""

In [2]:
import os
directory = "texts/individual/"
files = []
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        with open (directory+filename, "r") as myfile:
            data = myfile.read().replace('\n', ' ')
        files.append(filename)
    total = total + data

In [3]:
# Replace commas
total = total.replace("\xe2\x80\x99", "\'")
# Replace ...
total = total.replace("\xe2\x80\xa6", "...")
# Replace backslashes
total = total.replace("\\", "")
# Convert to lowercase
total = total.lower()

In [4]:
files

['the-cat-in-a-hat.txt',
 'horton-hears-a-who.txt',
 'ten-apples-up-on-top.txt',
 'wacky-wednesday.txt',
 'happy-birthday-to-you.txt',
 'if-i-ran-the-zoo.txt',
 'the-butter-battle-book.txt',
 'the-foot-book.txt',
 'oh-the-places-youll-go.txt',
 'the-sneetches.txt',
 'the-lorax.txt',
 'marvin-k-mooney-will-you-please-go-now.txt',
 'i-can-read-with-my-eyes-shut.txt',
 'wocket-in-my-pocket.txt',
 'hop-on-pop.txt',
 'how-the-grinch-stole-christmas.txt',
 'one-fish-two-fish-red-fish-blue-fish.txt',
 'mr-brown-can-moo-can-you.txt',
 'fox-in-socks.txt',
 'green-eggs-and-ham.txt',
 'in-a-people-house.txt']

In [5]:
total

'the sun did not shine. it was too wet to play. so we sat in the house all that cold, cold, wet day.  i sat there with sally. we sat there, we two. and i said, "how i wish we had something to do!"  too wet to go out and too cold to play ball. so we sat in the house. we did nothing at all.  so all we could do was to  sit! sit! sit! sit!  and we did not like it. not one little bit.  bump!  and then something went bump! how that bump made us jump!  we looked! then we saw him step in on the mat! we looked! and we saw him! the cat in the hat! and he said to us, "why do you sit there like that?" "i know it is wet and the sun is not sunny. but we can have lots of good fun that is funny!"  "i know some good games we could play," said the cat. "i know some new tricks," said the cat in the hat. "a lot of good tricks. i will show them to you. your mother will not mind at all if i do."  then sally and i did not know what to say. our mother was out of the house for the day.  but our fish said, "no!

## 2. Create Char Dictionary

In [6]:
def create_dict(texts):
    wl = {}
    for word in texts.split():
        if word not in wl:
            wl[word] = 1
        else:
            wl[word] += 1
    return wl

In [7]:
import operator
d = create_dict(total)
sorted_d = sorted(d.items(), key=lambda x: x[1], reverse=True)

In [8]:
print('Dictionary Size: %d' % len(sorted_d))

Dictionary Size: 3829


In [9]:
print('Dictionary:\n %s' % sorted_d)

Dictionary:
 [('the', 678), ('and', 543), ('a', 464), ('i', 385), ('you', 299), ('to', 275), ('in', 271), ('he', 227), ('of', 184), ('on', 166), ('with', 159), ('that', 153), ('not', 150), ('is', 142), ('can', 132), ('like', 122), ('my', 112), ('will', 112), ('they', 110), ('do', 105), ('we', 94), ('all', 94), ('are', 93), ('up', 92), ('them', 90), ('have', 87), ('then', 77), ('was', 76), ('as', 75), ('this', 74), ('for', 73), ('his', 73), ('but', 71), ('it', 70), ('or', 69), ('go', 68), ('so', 66), ('your', 66), ('what', 63), ('at', 59), ('one', 58), ('be', 56), ('said', 53), ('just', 53), ('no', 52), ('who', 50), ('things', 49), ('when', 47), ('some', 46), ('our', 46), ('would', 45), ('if', 40), ('out', 40), ('their', 39), ('very', 38), ("you'll", 38), ('here', 36), ('get', 36), ('had', 35), ('mr.', 35), ('from', 35), ('could', 35), ('how', 34), ('down', 34), ('more', 34), ('about', 32), ('said,', 31), ('there', 31), ("don't", 30), ('eat', 29), ('thing', 29), ('come', 29), ('by', 29)

## 3. Vectorize on Word Dict

In [10]:
SEQ_LENGTH = 25

In [12]:
words = []
for pair in sorted_d:
    words.append(pair[0])

In [14]:
n_to_word = {n:word for n, word in enumerate(words)}
word_to_n = {word:n for n, word in enumerate(words)}

In [17]:
import numpy as np
from keras.utils import np_utils
X = []
Y = []
totalsplt = total.split()
length = len(totalsplt)
for i in range(0, length - SEQ_LENGTH, 1):
    sequence = totalsplt[i:i + SEQ_LENGTH]
    label = totalsplt[i + SEQ_LENGTH]
    X.append([word_to_n[word] for word in sequence])
    Y.append(word_to_n[label])

In [18]:
X_mod = np.reshape(X, (len(X), SEQ_LENGTH, 1))
X_mod = X_mod / float(len(characters))
Y_mod = np_utils.to_categorical(Y)

In [19]:
X_mod.shape

(16413, 25, 1)

## 4. LSTM Model

In [20]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

In [21]:
model = Sequential()
model.add(LSTM(400, input_shape=(X_mod.shape[1], X_mod.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(400))
model.add(Dropout(0.2))
model.add(Dense(Y_mod.shape[1], activation = 'softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [22]:
model.fit(X_mod, Y_mod, epochs=1, batch_size=100)
model.save_weights('models/dr-seuss-baseline-word-rnn-400-0.2-400-0.2.h5')

Epoch 1/1
16413/16413 [==============================] - 208s - loss: 7.2768   


In [23]:
model.load_weights('models/dr-seuss-baseline-word-rnn-400-0.2-400-0.2.h5')

## 5. Generate Text

In [24]:
string_mapped = list(X[0])
full_string = [n_to_word[value] for value in string_mapped]
# generating characters
for i in range(75):
    x = np.reshape(string_mapped,(1,len(string_mapped), 1))
    x = x / float(len(characters))
    
    pred_index = np.argmax(model.predict(x, verbose=0))
    seq = [n_to_word[value] for value in string_mapped]
    full_string.append(n_to_word[pred_index])
    
    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]

In [26]:
#combining text
txt=""
for word in full_string:
    txt = txt + " " + word
print(txt)

 the sun did not shine. i
 the sun did not shine. it was too wet to play. so we sat in the house all that cold, cold, wet day. i sat the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the the
